# **LifeHarmony: An AI Recommender System for a Balanced Life**

## **Mapping Features for Model Input**

### Why Mapping is Needed:
To process user data in a machine-readable format, categorical variables such as marital status, occupation, and hobbies must be converted into numerical values.

### Mappings Defined:
- **Marital Status**: Single and Married are mapped to integers.
- **Occupation**: Each job type (e.g., Full-time, Freelancer) is assigned a unique number.
- **Personality**: Personality types are categorized into Extrovert, Introvert, and Ambivert.
- **Hobbies**: Options like Exercise, Art, and Writing are converted to integers.
- **Priority Mapping**: High/Medium/Low priorities are mapped to `2`, `1`, and `0`, respectively, for calculations.

These mappings ensure that our features can be effectively used in mathematical models and algorithms.


In [37]:
marital_status_mapping = {"Single": 0, "Married": 1}
occupation_mapping = {"Full-time": 0, "Part-time": 1, "Freelancer": 2, "Student": 3, "Unemployed": 4}
personality_mapping = {"Extrovert": 0, "Introvert": 1, "Ambivert": 2}
hobby_mapping = {"Exercise": 0, "Reading": 1, "Writing": 2, "Art": 3, "Socializing": 4}
priority_mapping = {"Low": 0, "Medium": 1, "High": 2}

life_features = ["Career", "Financial", "Spiritual", "Physical", "Intellectual", "Family", "Social", "Fun"]
gaps = {domain:0 for domain in life_features}
print(gaps)

{'Career': 0, 'Financial': 0, 'Spiritual': 0, 'Physical': 0, 'Intellectual': 0, 'Family': 0, 'Social': 0, 'Fun': 0}



## **Applying the Model with User Input**

### Step 1: Gather User Input
We ask the user to provide the following details:
   - **Personal Information**: Age, Gender, Marital Status, Occupation, Budget, Allocated Time, Personality, and Hobbies.
   - **Domain Ratings**: For each life domain (Career, Financial, Spiritual, etc.), users provide:
     - Current satisfaction rating (1-10).
     - Desired satisfaction rating (1-10).

### Purpose of Input:
This information helps determine the **gap** between a user’s current state and their goal. Domains with larger gaps are prioritized for recommendations.

### Example:
If a user rates their **Physical** satisfaction as `5` but wants it to be `8`, the gap is calculated as: `Gap = Desired Rating - Current Rating = 8 - 5 = 3`


In [38]:
age = 24
gender = "Female"
marital_status = "Single"
occupation = "Student"
budget = 3000
allocated_time = 3
personality = "Ambivert"
hobby = "Art"

In [39]:

ask_for_input = False
if ask_for_input:
    for domain in life_features:
        current = input(f"Enter your current rating for {domain} (1-10): ")
        goal = input(f"Enter your goal rating for {domain} (1-10): ")
        gaps[domain] = int(goal) - int(current)
else:
    gaps = {'Career': 3, 'Financial': 0, 'Spiritual': 1, 'Physical': 3, 'Intellectual': 2, 'Family': 2, 'Social': 2, 'Fun': 4}

print(gaps)

{'Career': 3, 'Financial': 0, 'Spiritual': 1, 'Physical': 3, 'Intellectual': 2, 'Family': 2, 'Social': 2, 'Fun': 4}



## **Normalizing Gaps and Assigning Priorities**

### Why Normalize?
Gaps across domains vary, and normalizing ensures fair comparisons by scaling all gaps relative to the largest one.

### Normalization Logic:
- Divide each gap by the maximum gap to get a value between `0` and `1`.
- Example: If the largest gap is `4` and another gap is `2`, the normalized value of `2` is `0.5`.

### Assigning Priorities:
- Domains are prioritized as **High**, **Medium**, or **Low** based on thresholds:
  - **Low**: Gap is in the bottom 25% of all normalized gaps.
  - **Medium**: Gap is between 25% and 66%.
  - **High**: Gap is in the top 33%.

If all gaps are equal, priorities are assigned alphabetically as a tie-breaking rule.



In [40]:
import numpy as np

# Normalize gaps (divide by max gap to get values between 0 and 1)
max_gap = max(abs(val) for val in gaps.values())
normalized_gaps = {key: abs(value) / max_gap for key, value in gaps.items()} if max_gap != 0 else {key: 0 for key in gaps}

# Assign priorities
if len(set(normalized_gaps.values())) == 1:  # All gaps are equal
    # Tie-breaking rule: Alphabetical order of domain names
    sorted_domains = sorted(normalized_gaps.keys())
    priorities = {
        key: "High" if key == sorted_domains[0] else "Medium" if key == sorted_domains[1] else "Low"
        for key in normalized_gaps
    }
else:
    # Assign priorities based on thresholds
    thresholds = np.percentile(list(normalized_gaps.values()), [25.00, 66.67])
    priorities = {
        key: "Low" if normalized_gaps[key] <= thresholds[0] else
                "Medium" if normalized_gaps[key] <= thresholds[1] else
                "High"
        for key in normalized_gaps
    }

print(priorities)


{'Career': 'High', 'Financial': 'Low', 'Spiritual': 'Low', 'Physical': 'High', 'Intellectual': 'Medium', 'Family': 'Medium', 'Social': 'Medium', 'Fun': 'High'}


## **Encoding the User State**

### Why Encode?
To compare the user's state with training data, all inputs need to be converted into a standardized numerical vector.

### How It's Done:
- Map each user input to its numerical representation (using the mappings defined earlier).
- Include priorities for all life domains.
- Example of an Encoded Vector:
```python
[24, 0, 3, 3000, 2, 3, 2, 1, 0, 0, 1, 2, 1, 1]

In [41]:
user_state_vector = [
    age,
    marital_status_mapping[marital_status],
    occupation_mapping[occupation],
    budget,
    personality_mapping[personality],
    hobby_mapping[hobby],
    *[priority_mapping[priorities[domain]] for domain in life_features],  # Map priorities for each life feature
]
print(user_state_vector)


[24, 0, 3, 3000, 2, 3, 2, 0, 0, 2, 1, 1, 1, 2]


## **Matching User State to Training Data**

### Purpose:
To find the most similar training states to the user’s encoded state, allowing for relevant recommendations to be made.

### How It Works:
1. **Nearest Neighbor Search**:
   - A **k-Nearest Neighbors (k-NN)** model is trained on the encoded training states.
   - For a given user’s state, the model finds the `k` most similar states based on a chosen similarity metric (e.g., cosine similarity, Euclidean distance).
   
2. **Weighted Aggregation**:
   - Neighbors closer to the user’s state are assigned higher weights.
   - The weights are inversely proportional to the distance from the user’s state.

### Example:
For a user with encoded state `[24, 0, 3, 3000, 2, 3, 2, 1, 0, 0, 1, 2, 1, 1]`:
- The k-NN model identifies the top `k` closest training states.
- Weights are computed for each neighbor to prioritize closer matches.


## **Making Recommendations**

### Using the Q-Table:
1. **Retrieve Q-Values**:
   - For each neighbor, extract the Q-values from the Q-Table. These values represent the model’s learned assessment of action quality for that state.
   
2. **Threshold-Based Filtering**:
   - Only actions with Q-values above a defined threshold (e.g., 70% of the maximum Q-value) are considered.
   - This ensures that only high-quality recommendations are included.

3. **Final Aggregation**:
   - Combine actions from all neighbors.
   - Ensure uniqueness by filtering out duplicate actions.

### Example Output:
For a user prioritizing **Career** and **Health**, the recommended actions might include:
- "Exercise regularly."
- "Update your resume."
- "Join professional networking events."

In [42]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Load training state vectors and Q-table
encoded_training_states = np.load("encoded_training_states.npy")  # Encoded training states
q_table = np.load("train_q_table.npy")  # Trained Q-table

# Load mappings
index_to_action = np.load("index_to_action.npy", allow_pickle=True).item()  # Index-to-action mapping

print(f"Encoded Training States: {encoded_training_states.shape[0]}")
print(f"Q-table Rows: {q_table.shape[0]}")

Encoded Training States: 9600
Q-table Rows: 9600


In [43]:
k = 3
threshold = 0.7

# User's encoded state
user_state_vector = np.array(user_state_vector).reshape(1, -1)

scaled_training_states = encoded_training_states 
user_state_vector_scaled = user_state_vector 


# Initialize and fit k-NN
knn = NearestNeighbors(n_neighbors=k, metric="cosine")
knn.fit(encoded_training_states)

# Find nearest neighbors
distances, indices = knn.kneighbors(user_state_vector)

# Retrieve optimal actions from Q-table
optimal_actions = []

for idx in indices[0]:
    state_q_values = q_table[idx]
    max_q_value = np.max(state_q_values)
    actions = [
        index_to_action[action]
        for action in np.where(state_q_values >= threshold * max_q_value)[0]
    ]
    optimal_actions.extend(actions)

# Get unique recommendations
unique_recommendations = list(set(optimal_actions))

# Display recommendations
print("Recommended Actions:")
for action in unique_recommendations:
    print(f"- {action}")


Recommended Actions:
- Join small, interest-based groups
- Consider getting a gym membership
- Visit art fairs with friends
- Engage in hobbies you enjoy
- Attend regular preventative medical checkups
- Watch your favorite shows
- Join group art classes
- Participate in online communities
- Visit family members regularly
- Allocate 1-3 hours a week into improving your career-related skills
- Try community hikes or dance classes
- Plan family gatherings
- Eat healthy foods


# **Conclusion**

The results of the **LifeHarmony AI Recommender System** demonstrate that the model's suggested recommendations align closely with expectations based on the given inputs and priorities. By leveraging a combination of **Reinforcement Learning** and **k-Nearest Neighbors (k-NN)**, the system successfully balances personalization and generalization, providing actionable suggestions tailored to individual user attributes and domain priorities.

### Key Takeaways:
- **Alignment with Expectations**:
  - The recommended actions for validation states closely match the true actions derived from user priorities. This indicates the model's ability to generalize well and align its outputs with logical expectations.
  
- **Dynamic Adjustments**:
  - The incorporation of weighted k-NN and threshold-based filtering ensures that recommendations remain relevant to the user's unique needs, avoiding unnecessary or irrelevant suggestions.

- **Practical Relevance**:
  - Actions like "Eat healthy foods" "Allocate 1-3 hours a week into improving your career-related skills" and "Engage in hobbies you enjoy" directly address high-priority domains while maintaining a balance across other areas.

### Final Thoughts:
This project highlights the potential of combining AI-driven methods with domain-specific logic to create a system that provides meaningful, user-focused recommendations. While there are areas for improvement, such as enhancing dynamic learning and incorporating collaborative filtering, the current implementation lays a solid foundation for expanding LifeHarmony into a robust and scalable tool for personal development and balance.
